In [1]:
import sys
dw_path = "/home/emmanuel/ecastillo/dev/delaware/10102024"
sys.path.append(dw_path)

In [2]:
from obspy import UTCDateTime
from delaware.core.client import CustomClient
import pandas as pd

provider = "USGS"
client =  CustomClient(provider)
# region = [-104.84329,-103.79942,31.39610,31.91505]
region = [-103.973638,-103.963891,31.607104,31.614540]

cat,picks,mag = client.get_custom_events(
                        minlatitude=region[2], maxlatitude=region[3], 
                        minlongitude=region[0], maxlongitude=region[1],
                        includeallorigins=True,
                        starttime=UTCDateTime(f"2023-01-01 15:26:30"),
                        endtime=UTCDateTime("2023-01-31 15:26:32"))
columns = []

cat.columns = cat.columns.get_level_values(1)
cat = cat.rename(columns={"latitude":"eq_latitude","longitude":"eq_longitude"})

picks["arrival_time"] = pd.to_datetime(picks["arrival_time"])
cat["origin_time"] = pd.to_datetime(cat["origin_time"])

cat_columns = ["ev_id","origin_time","eq_latitude","eq_longitude","magnitude"]
picks_columns = ["ev_id","station","phase_hint","arrival_time"]

cat = cat.drop_duplicates(ignore_index=True, keep="first")
picks = picks.dropna(subset="phase_hint")
picks= picks[picks["phase_hint"].isin(["P","S"])]
picks = picks.drop_duplicates(ignore_index=True, keep="first")

cat = cat[cat_columns]
picks[picks_columns]
print("### catalog ###\n",cat)
print("### picks ###\n",picks)

### catalog ###
             ev_id                origin_time  eq_latitude  eq_longitude  \
0  texnet2023agjl 2023-01-04 11:36:42.744357    31.610174   -103.972841   
1  texnet2023aoub 2023-01-09 02:02:10.199730    31.607540   -103.970274   
2  texnet2023aqgw 2023-01-09 21:37:08.142013    31.612600   -103.968682   
3  texnet2023aqgx 2023-01-09 21:37:51.061814    31.609739   -103.965308   
4  texnet2023bmii 2023-01-21 23:22:46.820996    31.610776   -103.964371   
5  texnet2023bmys 2023-01-22 07:44:04.425266    31.614333   -103.965866   
6  texnet2023bnkc 2023-01-22 13:24:55.312009    31.608334   -103.973371   
7  texnet2023bpct 2023-01-23 11:56:31.690517    31.608797   -103.972297   

   magnitude  
0        2.4  
1        2.7  
2        1.7  
3        1.8  
4        1.9  
5        1.7  
6        2.0  
7        2.4  
### picks ###
               ev_id network station location channel phase_hint  \
0    texnet2023agjl      4O    WB03       00     HHN          P   
1    texnet2023agjl    

In [3]:
import pandas as pd
stations_path = "/home/emmanuel/ecastillo/dev/delaware/10102024/data/stations/standard_stations.csv"
stations = pd.read_csv(stations_path)
stations_columns = ["network","station","latitude","longitude","elevation","x[km]","y[km]"]

picks_sta = pd.merge(picks,stations,on=["network","station"],how="inner")
new_cols = stations_columns
new_cols.remove("station")
picks_sta = picks_sta[picks_columns+new_cols]
picks_sta[0:10]

,ev_id,station,phase_hint,arrival_time,network,latitude,longitude,elevation,x[km],y[km]
0,texnet2023agjl,WB03,P,2023-01-04 11:36:44.736767,4O,31.610722,-103.968839,0.99400,-11573.758171,3712.319547
1,texnet2023agjl,WB03,S,2023-01-04 11:36:46.241872,4O,31.610722,-103.968839,0.99400,-11573.758171,3712.319547
2,texnet2023agjl,PB24,P,2023-01-04 11:36:45.016786,TX,31.619790,-104.017760,0.99700,-11579.204077,3713.504982
3,texnet2023agjl,PB24,S,2023-01-04 11:36:46.393550,TX,31.619790,-104.017760,0.99700,-11579.204077,3713.504982
4,texnet2023agjl,PB40,P,2023-01-04 11:36:44.970116,TX,31.641994,-103.933959,0.93500,-11569.875370,3716.407911
5,texnet2023agjl,PB40,S,2023-01-04 11:36:46.650234,TX,31.641994,-103.933959,0.93500,-11569.875370,3716.407911
6,texnet2023agjl,PB20,P,2023-01-04 11:36:45.471818,TX,31.628100,-104.061790,1.02200,-11584.105474,3714.591366
7,texnet2023agjl,PB20,S,2023-01-04 11:36:47.280279,TX,31.628100,-104.061790,1.02200,-11584.105474,3714.591366
8,texnet2023agjl,WB04,P,2023-01-04 11:36:45.320141,4O,31.530513,-103.950319,0.96189,-11571.696579,3701.839715
9,texnet2023agjl,WB04,S,2023-01-04 11:36:47.175271,4O,31.530513,-103.950319,0.96189,-11571.696579,3701.839715


In [4]:
picks_eq = pd.merge(picks_sta,cat,on=["ev_id"])
picks_eq[0:10]

,ev_id,station,phase_hint,arrival_time,network,latitude,longitude,elevation,x[km],y[km],origin_time,eq_latitude,eq_longitude,magnitude
0,texnet2023agjl,WB03,P,2023-01-04 11:36:44.736767,4O,31.610722,-103.968839,0.99400,-11573.758171,3712.319547,2023-01-04 11:36:42.744357,31.610174,-103.972841,2.4
1,texnet2023agjl,WB03,S,2023-01-04 11:36:46.241872,4O,31.610722,-103.968839,0.99400,-11573.758171,3712.319547,2023-01-04 11:36:42.744357,31.610174,-103.972841,2.4
2,texnet2023agjl,PB24,P,2023-01-04 11:36:45.016786,TX,31.619790,-104.017760,0.99700,-11579.204077,3713.504982,2023-01-04 11:36:42.744357,31.610174,-103.972841,2.4
3,texnet2023agjl,PB24,S,2023-01-04 11:36:46.393550,TX,31.619790,-104.017760,0.99700,-11579.204077,3713.504982,2023-01-04 11:36:42.744357,31.610174,-103.972841,2.4
4,texnet2023agjl,PB40,P,2023-01-04 11:36:44.970116,TX,31.641994,-103.933959,0.93500,-11569.875370,3716.407911,2023-01-04 11:36:42.744357,31.610174,-103.972841,2.4
5,texnet2023agjl,PB40,S,2023-01-04 11:36:46.650234,TX,31.641994,-103.933959,0.93500,-11569.875370,3716.407911,2023-01-04 11:36:42.744357,31.610174,-103.972841,2.4
6,texnet2023agjl,PB20,P,2023-01-04 11:36:45.471818,TX,31.628100,-104.061790,1.02200,-11584.105474,3714.591366,2023-01-04 11:36:42.744357,31.610174,-103.972841,2.4
7,texnet2023agjl,PB20,S,2023-01-04 11:36:47.280279,TX,31.628100,-104.061790,1.02200,-11584.105474,3714.591366,2023-01-04 11:36:42.744357,31.610174,-103.972841,2.4
8,texnet2023agjl,WB04,P,2023-01-04 11:36:45.320141,4O,31.530513,-103.950319,0.96189,-11571.696579,3701.839715,2023-01-04 11:36:42.744357,31.610174,-103.972841,2.4
9,texnet2023agjl,WB04,S,2023-01-04 11:36:47.175271,4O,31.530513,-103.950319,0.96189,-11571.696579,3701.839715,2023-01-04 11:36:42.744357,31.610174,-103.972841,2.4


In [5]:
from obspy.geodetics.base import gps2dist_azimuth

def get_distance_in_dataframe(data: pd.DataFrame, lat1_name: str, lon1_name: str,
                              lat2_name: str, lon2_name: str,columns=None):
    """
    Compute distances between two sets of latitude and longitude coordinates in a DataFrame.

    Args:
    - data (pd.DataFrame): Input DataFrame containing the latitude and longitude columns.
    - lat1_name (str): Name of the column containing the first set of latitudes.
    - lon1_name (str): Name of the column containing the first set of longitudes.
    - lat2_name (str): Name of the column containing the second set of latitudes.
    - lon2_name (str): Name of the column containing the second set of longitudes.
    - columns (list): Default:None means 'r','az','baz'. 3d List representing distance, azimuth y back azimuth

    Returns:
    - pd.DataFrame: DataFrame with additional columns 'r', 'az', 'baz' representing distance (in km),
      azimuth (degrees clockwise from north), and back azimuth (degrees clockwise from south), respectively.
    """
    if data.empty:
        return data
    
    data = data.reset_index(drop=True)
    computing_r = lambda x: gps2dist_azimuth(x[lat1_name], x[lon1_name],
                                             x[lat2_name], x[lon2_name])
    r = data.apply(computing_r, axis=1)
    
    if columns is None:
        columns = ["r", "az", "baz"]
        
    data[columns] = pd.DataFrame(r.tolist(), index=data.index)
    data[columns[0]] = data[columns[0]] / 1e3
    return data

picks_r_tt = get_distance_in_dataframe(data=picks_eq,lat1_name="latitude",
                                         lon1_name="longitude",lat2_name="eq_latitude",
                                         lon2_name="eq_longitude")

picks_r_tt["tt"] = picks_r_tt["arrival_time"] - picks_r_tt["origin_time"]
picks_r_tt["tt"] = picks_r_tt["tt"].dt.total_seconds()

picks_r_tt = picks_r_tt[picks_columns+["tt","r"]]
picks_r_tt.reset_index(inplace=True,drop=True)
picks_r_tt

/tmp/ipykernel_32096/3611965819.py:24: DeprecationWarning: `alltrue` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `all` instead.
  computing_r = lambda x: gps2dist_azimuth(x[lat1_name], x[lon1_name],


,ev_id,station,phase_hint,arrival_time,tt,r
0,texnet2023agjl,WB03,P,2023-01-04 11:36:44.736767,1.992410,0.384576
1,texnet2023agjl,WB03,S,2023-01-04 11:36:46.241872,3.497515,0.384576
2,texnet2023agjl,PB24,P,2023-01-04 11:36:45.016786,2.272429,4.393534
3,texnet2023agjl,PB24,S,2023-01-04 11:36:46.393550,3.649193,4.393534
4,texnet2023agjl,PB40,P,2023-01-04 11:36:44.970116,2.225759,5.104466
...,...,...,...,...,...,...
323,texnet2023bpct,SA07,S,2023-01-23 11:56:46.178846,14.488329,46.855403
324,texnet2023bpct,PB29,P,2023-01-23 11:56:41.618325,9.927808,53.835521
325,texnet2023bpct,PB29,S,2023-01-23 11:56:48.223645,16.533128,53.835521
326,texnet2023bpct,PB37,P,2023-01-23 11:56:41.358077,9.667560,54.124134


In [6]:
picks_ready = picks_r_tt.pivot(index=["ev_id", "station", "r"], columns="phase_hint", 
                      values=["arrival_time", "tt"]).reset_index()
# Flatten multi-level columns
picks_ready.columns = ['_'.join(col).strip('_') for col in picks_ready.columns.values]
picks_ready.dropna(subset=["tt_P","tt_S"],inplace=True)
picks_ready.sort_values(by="tt_P",inplace=True,ignore_index=True)
picks_ready

,ev_id,station,r,arrival_time_P,arrival_time_S,tt_P,tt_S
0,texnet2023bmys,WB03,0.489826,2023-01-22 07:44:06.046913,2023-01-22 07:44:07.326684,1.621647,2.901418
1,texnet2023aqgw,WB03,0.208871,2023-01-09 21:37:09.873069,2023-01-09 21:37:11.272263,1.731056,3.13025
2,texnet2023bmii,WB03,0.424002,2023-01-21 23:22:48.561550,2023-01-21 23:22:49.894500,1.740554,3.073504
3,texnet2023aqgx,WB03,0.352335,2023-01-09 21:37:52.815583,2023-01-09 21:37:54.230705,1.753769,3.168891
4,texnet2023bmys,PB40,4.309264,2023-01-22 07:44:06.239755,2023-01-22 07:44:07.642244,1.814489,3.216978
...,...,...,...,...,...,...,...
150,texnet2023aoub,PB37,54.342816,2023-01-09 02:02:19.880499,2023-01-09 02:02:26.406067,9.680769,16.206337
151,texnet2023aoub,PB29,54.060459,2023-01-09 02:02:19.971765,2023-01-09 02:02:26.619023,9.772035,16.419293
152,texnet2023agjl,PB37,54.039352,2023-01-04 11:36:52.587893,2023-01-04 11:36:58.631651,9.843536,15.887294
153,texnet2023agjl,PB29,53.740718,2023-01-04 11:36:52.634563,2023-01-04 11:36:58.724991,9.890206,15.980634


In [16]:
import numpy as np

def loss_function(particles, r, tp_obs, ts_obs):
    z_guess = particles[:, 0]
    vp_guess = particles[:, 1]
    vs_guess = particles[:, 2]

    if np.any(vp_guess == 0):
        raise Exception("vp_guess must not contain any zeros")
    if np.any(vs_guess == 0):
        raise Exception("vs_guess must not contain any zeros")

    tp_pred = np.sqrt(r**2 + z_guess**2) / vp_guess
    ts_pred = np.sqrt(r**2 + z_guess**2) / vs_guess
    loss = ((tp_pred - tp_obs)**2) + ((ts_pred - ts_obs)**2)
    return loss

def vd_pso2(cost_func,
           picks,
           bounds,
           max_iter=100,w=0.5,c1=0.8,c2=0.9):
    
    n_particles = len(picks)
    dim = bounds.shape[-1]
    
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, dim))
    velocities = np.zeros((n_particles, dim))
    args = {col: picks[col].to_numpy() for col in ["r",'tt_P','tt_S']}
    
    # Initialize the particles and velocities
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, 3))
    velocities = np.zeros((n_particles, 3))

    # Initialize the best positions and best costs
    best_positions = particles.copy()
    best_costs = cost_func(
                            particles=particles,
                            r=args["r"],
                            tp_obs=args["tt_P"],
                            ts_obs=args["tt_S"]
                        )

    # Initialize the global best position and global best cost
    global_best_position = particles[0].copy()
    global_best_cost = best_costs[0]

    # Perform the optimization
    for i in range(max_iter):
        # Update the velocities
        r1 = np.random.rand(n_particles, 3) #Random matrix used to compute the cognitive component of the veocity update
        r2 = np.random.rand(n_particles, 3) #Random matrix used to compute the social component of the veocity update


        #Cognitive component is calculated by taking the difference between the
        #particle's current position and its best personal position found so far,
        #and then multiplying it by a random matrix r1 and a cognitive acceleration coefficient c1.
        cognitive_component = c1 * r1 * (best_positions - particles)

        #The social component represents the particle's tendency to move towards the
        #global best position found by the swarm. It is calculated by taking the
        #difference between the particle's current position and the global best position
        # found by the swarm, and then multiplying it by a random matrix r2 and a
        #social acceleration coefficient c2.
        social_component = c2 * r2 * (global_best_position - particles)

        #The new velocity of the particle is computed by adding the current velocity
        #to the sum of the cognitive and social components, multiplied by the inertia
        #weight w. The new velocity is then used to update the position of the
        #particle in the search space.
        velocities = w * velocities + cognitive_component + social_component

        # Update the particles
        particles += velocities

        # Enforce the bounds of the search space
        particles = np.clip(particles, bounds[0], bounds[1])

        # Evaluate the objective function
        costs = cost_func(
                            particles=particles,
                            r=args["r"],
                            tp_obs=args["tt_P"],
                            ts_obs=args["tt_S"]
                        )

        # Update the best positions and best costs
        is_best = costs < best_costs
        best_positions[is_best] = particles[is_best]
        best_costs[is_best] = costs[is_best]

        # Update the global best position and global best cost
        global_best_index = np.argmin(best_costs)
        global_best_position = best_positions[global_best_index].copy()
        global_best_cost = best_costs[global_best_index]

        # Print the progress
        if (i+1)%10 == 0:
            print(f'Iteration {i+1}: Best Cost = {global_best_cost:.6f} | sol:{global_best_position}')
    
    return global_best_position, global_best_cost



cost_func = loss_function
bounds = np.array([[0,2.5,1.2],[10,6,3.5]])

# Run the PSO algorithm on the Rastrigin function
solution, fitness = vd_pso2(cost_func=cost_func,picks=picks_ready,
                            bounds=bounds,max_iter=500
                            )

solution, fitness

Iteration 10: Best Cost = 0.000182 | sol:[6.98127608 4.38395277 2.56130202]
Iteration 20: Best Cost = 0.000000 | sol:[6.98431081 4.38194628 2.55159387]
Iteration 30: Best Cost = 0.000000 | sol:[6.98419852 4.38218305 2.55178092]
Iteration 40: Best Cost = 0.000000 | sol:[6.98419693 4.3821913  2.5517721 ]
Iteration 50: Best Cost = 0.000000 | sol:[6.98419692 4.38219133 2.55177212]
Iteration 60: Best Cost = 0.000000 | sol:[6.98419692 4.38219133 2.55177212]
Iteration 70: Best Cost = 0.000000 | sol:[6.98419692 4.38219133 2.55177212]
Iteration 80: Best Cost = 0.000000 | sol:[6.98419692 4.38219133 2.55177212]
Iteration 90: Best Cost = 0.000000 | sol:[6.98419692 4.38219133 2.55177212]
Iteration 100: Best Cost = 0.000000 | sol:[6.98419692 4.38219133 2.55177212]
Iteration 110: Best Cost = 0.000000 | sol:[6.98419692 4.38219133 2.55177212]
Iteration 120: Best Cost = 0.000000 | sol:[6.98419692 4.38219133 2.55177212]
Iteration 130: Best Cost = 0.000000 | sol:[6.98419692 4.38219133 2.55177212]
Iteratio

(array([6.98419692, 4.38219133, 2.55177212]), 3.24234911394997e-08)

In [19]:
import numpy as np

def loss_function(particles, r, tp_obs, ts_obs):
    z_guess = particles[:, 0]
    vp_guess = particles[:, 1]
    vs_guess = particles[:, 2]

    if np.any(vp_guess == 0):
        raise Exception("vp_guess must not contain any zeros")
    if np.any(vs_guess == 0):
        raise Exception("vs_guess must not contain any zeros")

    tp_pred = np.sqrt(r**2 + z_guess**2) / vp_guess
    ts_pred = np.sqrt(r**2 + z_guess**2) / vs_guess
    loss = ((tp_pred - tp_obs)**2) + ((ts_pred - ts_obs)**2)
    return loss

def gradient(loss_func, particles, r, tp_obs, ts_obs, epsilon=1e-6):
    """Calculate gradients for the loss function w.r.t. particles."""
    gradients = np.zeros_like(particles)
    for i in range(particles.shape[1]):
        # Perturb each parameter by a small epsilon and calculate the gradient
        particles_plus = particles.copy()
        particles_plus[:, i] += epsilon
        loss_plus = loss_func(particles_plus, r, tp_obs, ts_obs)
        
        particles_minus = particles.copy()
        particles_minus[:, i] -= epsilon
        loss_minus = loss_func(particles_minus, r, tp_obs, ts_obs)
        
        # Compute the gradient (difference between perturbed losses)
        gradients[:, i] = (loss_plus - loss_minus) / (2 * epsilon)
        
    return gradients

def vd_gradient_descent(cost_func, picks, bounds, max_iter=100, lr=0.01):
    n_particles = len(picks)
    dim = bounds.shape[-1]
    
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, dim))
    args = {col: picks[col].to_numpy() for col in ["r", 'tt_P', 'tt_S']}
    
    # Initialize the best position and best cost
    costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])
    best_cost = np.min(costs)
    best_position = particles[np.argmin(costs)].copy()

    # Perform gradient descent optimization
    for i in range(max_iter):
        # Compute gradients of the loss function
        gradients = gradient(cost_func, particles, args["r"], args["tt_P"], args["tt_S"])
        
        # Update particles using gradient descent
        particles -= lr * gradients

        # Enforce the bounds of the search space
        particles = np.clip(particles, bounds[0], bounds[1])

        # Evaluate the new costs
        costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])

        # Update best position if new cost is lower
        current_best_cost = np.min(costs)
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_position = particles[np.argmin(costs)].copy()

        # Print progress
        if (i + 1) % 10 == 0:
            print(f'Iteration {i+1}: Best Cost = {best_cost} | sol:{best_position}')

    return best_position, best_cost

# Example of how to call the Gradient Descent-based optimization function
bounds = np.array([[0, 2.5, 1.2], [10, 6, 3.5]])  # bounds for z_guess, vp_guess, vs_guess

# Call the function with your data (picks_ready should be a DataFrame or similar)
solution, fitness = vd_gradient_descent(cost_func=loss_function, picks=picks_ready, bounds=bounds, max_iter=500, lr=0.01)

print("Best Solution:", solution)
print("Best Fitness:", fitness)

Iteration 10: Best Cost = 0.0040171693486686835 | sol:[0.35202443 5.59415368 3.33917636]
Iteration 20: Best Cost = 0.0014451056450336956 | sol:[0.3520423  5.5806869  3.33231352]
Iteration 30: Best Cost = 0.0006386951896920184 | sol:[0.35205155 5.57167936 3.33215271]
Iteration 40: Best Cost = 0.00011414755378041958 | sol:[3.67213158 2.70177254 1.64368502]
Iteration 50: Best Cost = 1.761774811593351e-05 | sol:[3.6727469  2.70136373 1.64072215]
Iteration 60: Best Cost = 5.084736753317611e-06 | sol:[3.67301342 2.70102127 1.63970954]
Iteration 70: Best Cost = 2.8483026209970286e-06 | sol:[3.67315203 2.70072886 1.63937097]
Iteration 80: Best Cost = 2.0184260051066544e-06 | sol:[3.67324032 2.70047722 1.63926293]
Iteration 90: Best Cost = 1.4966579074609347e-06 | sol:[3.67330586 2.70025999 1.63923288]
Iteration 100: Best Cost = 7.02123166383136e-07 | sol:[0.55420623 5.4344803  3.36304404]
Iteration 110: Best Cost = 1.78304058869475e-07 | sol:[0.55420647 5.43425189 3.36304404]
Iteration 120: Be

In [22]:
import numpy as np

def loss_function(particles, r, tp_obs, ts_obs):
    z_guess = particles[:, 0]
    vp_guess = particles[:, 1]
    vs_guess = particles[:, 2]

    if np.any(vp_guess == 0):
        raise Exception("vp_guess must not contain any zeros")
    if np.any(vs_guess == 0):
        raise Exception("vs_guess must not contain any zeros")

    tp_pred = np.sqrt(r**2 + z_guess**2) / vp_guess
    ts_pred = np.sqrt(r**2 + z_guess**2) / vs_guess
    loss = ((tp_pred - tp_obs)**2) + ((ts_pred - ts_obs)**2)
    return loss

def vd_adam(cost_func, picks, bounds, max_iter=100, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
    n_particles = len(picks)
    dim = bounds.shape[-1]
    
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, dim))
    args = {col: picks[col].to_numpy() for col in ["r", 'tt_P', 'tt_S']}
    
    # Initialize moment estimates
    m = np.zeros_like(particles)  # First moment (mean of gradients)
    v = np.zeros_like(particles)  # Second moment (uncentered variance of gradients)
    t = 0  # Time step (for bias correction)

    # Evaluate initial cost
    costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])
    best_cost = np.min(costs)
    best_position = particles[np.argmin(costs)].copy()

    # Perform Adam optimization
    for i in range(max_iter):
        t += 1  # Increment time step

        # Compute gradients (numerical approximation)
        gradients = gradient(cost_func, particles, args["r"], args["tt_P"], args["tt_S"])

        # Update moment estimates (bias correction applied to m and v)
        m = beta1 * m + (1 - beta1) * gradients  # Update first moment
        v = beta2 * v + (1 - beta2) * gradients**2  # Update second moment

        # Bias correction (to counteract initialization bias)
        m_hat = m / (1 - beta1**t)
        v_hat = v / (1 - beta2**t)

        # Update particles using Adam's update rule
        particles -= lr * m_hat / (np.sqrt(v_hat) + epsilon)

        # Enforce the bounds of the search space
        particles = np.clip(particles, bounds[0], bounds[1])

        # Evaluate new costs
        costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])

        # Update best position if new cost is lower
        current_best_cost = np.min(costs)
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_position = particles[np.argmin(costs)].copy()

        # Print progress
        if (i + 1) % 10 == 0:
            print(f'Iteration {i+1}: Best Cost = {best_cost:.6f} | sol:{best_position}')

    return best_position, best_cost

def gradient(loss_func, particles, r, tp_obs, ts_obs, epsilon=1e-6):
    """Calculate gradients for the loss function w.r.t. particles."""
    gradients = np.zeros_like(particles)
    for i in range(particles.shape[1]):
        # Perturb each parameter by a small epsilon and calculate the gradient
        particles_plus = particles.copy()
        particles_plus[:, i] += epsilon
        loss_plus = loss_func(particles_plus, r, tp_obs, ts_obs)
        
        particles_minus = particles.copy()
        particles_minus[:, i] -= epsilon
        loss_minus = loss_func(particles_minus, r, tp_obs, ts_obs)
        
        # Compute the gradient (difference between perturbed losses)
        gradients[:, i] = (loss_plus - loss_minus) / (2 * epsilon)
        
    return gradients

# Example of how to call the Adam-based optimization function
bounds = np.array([[0, 2.5, 1.2], [10, 6, 3.5]])  # bounds for z_guess, vp_guess, vs_guess

# Call the function with your data (picks_ready should be a DataFrame or similar)
solution, fitness = vd_adam(cost_func=loss_function, picks=picks_ready, bounds=bounds, max_iter=500, lr=0.001)

print("Best Solution:", solution)
print("Best Fitness:", fitness)

Iteration 10: Best Cost = 0.000857 | sol:[9.66508209 4.84756867 2.8829762 ]
Iteration 20: Best Cost = 0.000546 | sol:[9.65689238 4.85710156 2.88019287]
Iteration 30: Best Cost = 0.000320 | sol:[9.64929916 4.86587515 2.87558571]
Iteration 40: Best Cost = 0.000170 | sol:[9.64185673 4.8735787  2.8747677 ]
Iteration 50: Best Cost = 0.000082 | sol:[9.63574619 4.87999047 2.87356392]
Iteration 60: Best Cost = 0.000035 | sol:[9.63110403 4.88503963 2.87168905]
Iteration 70: Best Cost = 0.000013 | sol:[9.62748643 4.8887888  2.87105743]
Iteration 80: Best Cost = 0.000004 | sol:[9.62502454 4.89138803 2.87038637]
Iteration 90: Best Cost = 0.000001 | sol:[9.62345905 4.89305415 2.86988916]
Iteration 100: Best Cost = 0.000000 | sol:[9.62252021 4.89402322 2.86972965]
Iteration 110: Best Cost = 0.000000 | sol:[9.62206452 4.89451407 2.86955517]
Iteration 120: Best Cost = 0.000000 | sol:[9.62198567 4.89459671 2.86953573]
Iteration 130: Best Cost = 0.000000 | sol:[9.62198567 4.89459671 2.86953573]
Iteratio